In [2]:
import duckdb
import os

# Close the DuckDB connection
con.close()

# Specify the path to the DuckDB database file
database_file = 'air_flight_database.db'

# Check if the database file exists
if os.path.exists(database_file):
    # If it exists, delete the database file
    os.remove(database_file)
    print("Database file deleted successfully.")
else:
    print("Database file does not exist.")


Database file deleted successfully.


In [3]:
import duckdb
import pandas as pd
import os

# Connect to DuckDB
con = duckdb.connect(database='air_flight_database.db', read_only=False)

In [4]:
# Load data from flights.csv into a DataFrame
df_flights = pd.read_csv('flights.csv')

# Clean and transform data and create the bronze table
con.execute('''
CREATE TABLE bronze AS
SELECT
    id, year, month, day, dep_time, sched_dep_time, dep_delay, arr_time, sched_arr_time, arr_delay,
    carrier, flight, tailnum, origin, dest, air_time, distance, hour, minute, time_hour, name
FROM df_flights
WHERE dep_delay IS NOT NULL AND arr_delay IS NOT NULL
''')

# Register the bronze table
con.register('bronze', df_flights)

# Verify the number of rows in the bronze table
row_count = con.execute('SELECT COUNT(*) FROM bronze').fetchone()[0]
print("Number of rows in bronze table:", row_count)

Number of rows in bronze table: 336776


In [5]:
con.execute('SELECT * FROM bronze LIMIT 5').fetchdf()

,id,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,...,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,name
0,0,2013,1,1,517.0,515,2.0,830.0,819,11.0,...,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00,United Air Lines Inc.
1,1,2013,1,1,533.0,529,4.0,850.0,830,20.0,...,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00,United Air Lines Inc.
2,2,2013,1,1,542.0,540,2.0,923.0,850,33.0,...,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00,American Airlines Inc.
3,3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,...,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01 05:00:00,JetBlue Airways
4,4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,...,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01 06:00:00,Delta Air Lines Inc.


### Performing Data Cleaning in Silver Layer


In [6]:
# Count the number of rows with null values in any column before cleaning
null_count_before = con.execute('''
SELECT COUNT(*) 
FROM bronze
WHERE id IS NULL OR year IS NULL OR month IS NULL OR day IS NULL OR dep_time IS NULL OR sched_dep_time IS NULL OR
      dep_delay IS NULL OR arr_time IS NULL OR sched_arr_time IS NULL OR arr_delay IS NULL OR carrier IS NULL OR
      flight IS NULL OR tailnum IS NULL OR origin IS NULL OR dest IS NULL OR air_time IS NULL OR distance IS NULL OR
      hour IS NULL OR minute IS NULL OR time_hour IS NULL OR name IS NULL
''').fetchone()[0]

print("Number of rows with null values before cleaning:", null_count_before)

# Count the number of duplicate rows before cleaning
duplicate_count_before = con.execute('''
SELECT COUNT(*) - COUNT(DISTINCT id) 
FROM bronze
''').fetchone()[0]

print("Number of duplicate rows before cleaning:", duplicate_count_before)



Number of rows with null values before cleaning: 9430
Number of duplicate rows before cleaning: 0


In [7]:
# creating new table where all non-null values will be saved. 
# as duck db doesn't allows deletion and renaming of the tables in the database- 

# Create a new table final_bronze with non-null values
con.execute('''
CREATE TABLE final_bronze AS
SELECT *
FROM bronze
WHERE id IS NOT NULL AND year IS NOT NULL AND month IS NOT NULL AND day IS NOT NULL AND dep_time IS NOT NULL AND sched_dep_time IS NOT NULL AND
      dep_delay IS NOT NULL AND arr_time IS NOT NULL AND sched_arr_time IS NOT NULL AND arr_delay IS NOT NULL AND carrier IS NOT NULL AND
      flight IS NOT NULL AND tailnum IS NOT NULL AND origin IS NOT NULL AND dest IS NOT NULL AND air_time IS NOT NULL AND distance IS NOT NULL AND
      hour IS NOT NULL AND minute IS NOT NULL AND time_hour IS NOT NULL AND name IS NOT NULL
''')


## For Gold Layer
### Creating fact and dimensions table 

#### Creating dimensions table first

In [8]:
# Create dimension tables
con.execute('''
CREATE TABLE dim_airports (
    airport_id INTEGER PRIMARY KEY,
    airport_name VARCHAR
)
''')
con.execute('''
CREATE TABLE dim_flight_schedule (
    id INTEGER PRIMARY KEY,
    dep_time INTEGER,
    sched_dep_time INTEGER,
    dep_delay INTEGER,
    arr_time INTEGER,
    sched_arr_time INTEGER,
    arr_delay INTEGER
)
''')
con.execute('''
CREATE TABLE dim_airline (
    airline_id INTEGER PRIMARY KEY,
    carrier_code VARCHAR,
    airline_name VARCHAR
)
''')
con.execute('''
CREATE TABLE dim_aircraft (
    aircraft_id INTEGER PRIMARY KEY,
    tailnum VARCHAR
)
''')


#### now populate the dimensions table - 

In [9]:

# Populate dimension table dim_airports
con.execute('''
INSERT INTO dim_airports (airport_name)
SELECT origin AS airport_name FROM final_bronze WHERE origin IS NOT NULL
''')

# dim_flight_schedule
con.execute('''
INSERT INTO dim_flight_schedule (id, dep_time, sched_dep_time, dep_delay, arr_time, sched_arr_time, arr_delay)
SELECT id, dep_time, sched_dep_time, dep_delay, arr_time, sched_arr_time, arr_delay FROM final_bronze
''')

# dim_airline
con.execute('''
INSERT INTO dim_airline (carrier_code, airline_name)
VALUES
    ('WN', 'Southwest Airlines Co.'), ('DL', 'Delta Air Lines Inc.'), ('AA', 'American Airlines Inc.'),
    ('UA', 'United Air Lines Inc.'),  ('B6', 'JetBlue Airways'), ('AS', 'Alaska Airlines Inc.'),
    ('NK', 'Spirit Air Lines'), ('G4', 'Allegiant Air'), ('F9', 'Frontier Airlines Inc.'),
    ('HA', 'Hawaiian Airlines Inc.'), ('SY', 'Sun Country Airlines d/b/a MN Airlines'),
    ('VX', 'Virgin America')
''')

# dim_aircraft
con.execute('''
INSERT INTO dim_aircraft (tailnum)
SELECT DISTINCT tailnum FROM final_bronze
''')


### Creating the fact table - 

In [14]:
# Create fact table without foreign key constraints
con.execute('''
CREATE TABLE fact_flights (
    flight_id INTEGER,
    flight_date DATE,
    carrier_code VARCHAR,
    origin_airport_id INTEGER,
    dest_airport_id INTEGER,
    dep_delay INTEGER,
    arr_delay INTEGER,
    air_time INTEGER,
    distance INTEGER
)
''')

# Populate fact table
con.execute('''
INSERT INTO fact_flights (flight_date, carrier_code, origin_airport_id, dest_airport_id, dep_delay, arr_delay, air_time, distance)
SELECT
    CAST(time_hour AS DATE) AS flight_date,
    carrier AS carrier_code,
    (SELECT airport_id FROM dim_airports WHERE airport_name = origin) AS origin_airport_id,
    (SELECT airport_id FROM dim_airports WHERE airport_name = dest) AS dest_airport_id,
    dep_delay,
    arr_delay,
    air_time,
    distance
FROM final_bronze
''')


### Exporting all the data as csv

In [15]:
# Export dimension tables to CSV
dim_airports_df = con.execute("SELECT * FROM dim_airports").fetchdf()
dim_airports_df.to_csv("dim_airports.csv", index=False)

dim_flight_schedule_df = con.execute("SELECT * FROM dim_flight_schedule").fetchdf()
dim_flight_schedule_df.to_csv("dim_flight_schedule.csv", index=False)

dim_airline_df = con.execute("SELECT * FROM dim_airline").fetchdf()
dim_airline_df.to_csv("dim_airline.csv", index=False)

dim_aircraft_df = con.execute("SELECT * FROM dim_aircraft").fetchdf()
dim_aircraft_df.to_csv("dim_aircraft.csv", index=False)

# Export fact table to CSV
fact_flights_df = con.execute("SELECT * FROM fact_flights").fetchdf()
fact_flights_df.to_csv("fact_flights.csv", index=False)

print("Data exported successfully")

Data exported successfully
